In [8]:
import re
import pandas as pd

def extract_params_from_file_name(df: pd.DataFrame):
    df['maxNewTokensFactor'] = df['file'].apply(lambda x: re.search(r'maxNewTokensFactor(\d+)', x).group(1))
    df['nShotsInference'] = df['file'].apply(lambda x: re.search(r'nShotsInference(\d+)', x).group(1))
    #df['layer'] = df['file'].apply(lambda x: re.search(r'adapters_(\s+)', x).group(1))
    df['model'] = df['file'].apply(lambda x: str(x.split('_adapters_')[0].split('nShotsInference')[1][2:]))
    df['training_params_string'] = df['file'].apply(lambda x: x.split('adapters_')[1])
    df['nbit'] = df['training_params_string'].apply(lambda x: int(x.split('_')[1]))
    df['bnb_4bit_compute_dtype'] = df['training_params_string'].apply(lambda x: x.split('_')[2])
    df['r'] = df['training_params_string'].apply(lambda x: int(x.split('_')[3]))
    df['lora_alpha'] = df['training_params_string'].apply(lambda x: int(x.split('_')[4]))
    df['lora_dropout'] = df['training_params_string'].apply(lambda x: float(x.split('_')[5]))
    df['gradient_accumulation_steps'] = df['training_params_string'].apply(lambda x: int(x.split('_')[6]))
    df['learning_rate'] = df['training_params_string'].apply(lambda x: float(x.split('_')[7].split('.')[0]))
    df = df.drop(columns=['training_params_string', 'file'])
    return df


In [8]:
llama_13B_4bit_base_wordsLevelTrue_evaluation = pd.read_csv("data/evaluation_results/llama_13B_4bit_base_wordsLevelTrue_evaluation.csv")
llama_13B_8bit_base_wordsLevelTrue_evaluation = pd.read_csv("data/evaluation_results/llama_13B_8bit_base_wordsLevelTrue_evaluation.csv")
llama_7B_4bit_base_wordsLevelTrue_evaluation = pd.read_csv("data/evaluation_results/llama_7B_4bit_base_wordsLevelTrue_evaluation.csv")
llama_7B_8bit_base_wordsLevelTrue_evaluation = pd.read_csv("data/evaluation_results/llama_7B_8bit_base_wordsLevelTrue_evaluation.csv")
llama_7B_8bit_wordsLevelTrue_evaluation = pd.read_csv("data/evaluation_results/llama_7B_8bit_wordsLevelTrue_evaluation.csv")
mistral_4bit_base_wordsLevelTrue_evaluation = pd.read_csv("data/evaluation_results/mistral_4bit_base_wordsLevelTrue_evaluation.csv")
mistral_4bit_wordsLevelTrue_evaluation = pd.read_csv("data/evaluation_results/mistral_4bit_wordsLevelTrue_evaluation.csv")
mistral_noInstr_4bit_wordsLevelTrue_evaluation = pd.read_csv("data/evaluation_results/mistral_noInstr_4bit_wordsLevelTrue_evaluation.csv")

# llama_13B_4bit_base_wordsLevelTrue_evaluation = extract_params_from_file_name(llama_13B_4bit_base_wordsLevelTrue_evaluation)
# llama_13B_8bit_base_wordsLevelTrue_evaluation = extract_params_from_file_name(llama_13B_8bit_base_wordsLevelTrue_evaluation)
# llama_7B_4bit_base_wordsLevelTrue_evaluation = extract_params_from_file_name(llama_7B_4bit_base_wordsLevelTrue_evaluation)
# llama_7B_8bit_base_wordsLevelTrue_evaluation = extract_params_from_file_name(llama_7B_8bit_base_wordsLevelTrue_evaluation)
# llama_7B_8bit_wordsLevelTrue_evaluation = extract_params_from_file_name(llama_7B_8bit_wordsLevelTrue_evaluation)
# mistral_4bit_base_wordsLevelTrue_evaluation = extract_params_from_file_name(mistral_4bit_base_wordsLevelTrue_evaluation)
# mistral_4bit_wordsLevelTrue_evaluation = extract_params_from_file_name(mistral_4bit_wordsLevelTrue_evaluation)
# mistral_noInstr_4bit_wordsLevelTrue_evaluation = extract_params_from_file_name(mistral_noInstr_4bit_wordsLevelTrue_evaluation)

res = pd.concat([llama_13B_4bit_base_wordsLevelTrue_evaluation, llama_13B_8bit_base_wordsLevelTrue_evaluation,
           llama_7B_4bit_base_wordsLevelTrue_evaluation, llama_7B_8bit_base_wordsLevelTrue_evaluation,
           llama_7B_8bit_wordsLevelTrue_evaluation, mistral_4bit_base_wordsLevelTrue_evaluation,
           mistral_4bit_wordsLevelTrue_evaluation, mistral_noInstr_4bit_wordsLevelTrue_evaluation])#.to_csv('data/evaluation_results/joint_results.csv')

In [13]:
l = '3B'
int(l)

ValueError: invalid literal for int() with base 10: '3B'

In [11]:
import glob
import pandas as pd
import re

def extract_params_from_file_name(df: pd.DataFrame):

    def adjust_model_size_format(model_size_str):
        try:
            int(model_size_str)
        except ValueError:
            model_size_str = model_size_str.replace('b', '').replace('B', '')



    df['model_type'] = df['file'].apply(lambda x: str(x.split('/')[1]))
    df['model_configurations'] = df['file'].apply(lambda x: str(x.split('/')[2]))
    if df['model_type'][0] == 'mistral' or df['model_type'][0] == 'zefiro':
        df['model_size'] = 7
        df['quantization'] = df['model_configurations'].apply(lambda x: str(x.split('_')[0]))
        try:
            df['fine_tuning'] = df['model_configurations'].apply(lambda x: str(x.split('_')[1]))
        except IndexError:
            df['fine_tuning'] = 'FT'
    else:
        df['model_size'] = df['model_configurations'].apply(lambda x: str(x.split('_')[0]))
        df['quantization'] = df['model_configurations'].apply(lambda x: str(x.split('_')[1]))
        df['fine_tuning'] = df['model_configurations'].apply(lambda x: str(x.split('_')[2]))

    df['maxNewTokensFactor'] = df['file'].apply(lambda x: re.search(r'maxNewTokensFactor(\d+)', x).group(1))
    df['nShotsInference'] = df['file'].apply(lambda x: re.search(r'nShotsInference(\d+)', x).group(1))
    #
    # df['layer'] = df['file'].apply(lambda x: re.search(r'adapters_(\s+)', x).group(1))
    df['model'] = df['file'].apply(lambda x: str(x.split('_adapters_')[0].split('nShotsInference')[1][2:].replace('.csv', '')))
    if df['fine_tuning'][0] == 'FT':
        df['training_params_string'] = df['file'].apply(lambda x: x.split('adapters_')[1])
        df['nbit'] = df['training_params_string'].apply(lambda x: x.split('_')[1])
        df['bnb_4bit_compute_dtype'] = df['training_params_string'].apply(lambda x: x.split('_')[2])
        df['r'] = df['training_params_string'].apply(lambda x: int(x.split('_')[3]))
        df['lora_alpha'] = df['training_params_string'].apply(lambda x: int(x.split('_')[4]))
        df['lora_dropout'] = df['training_params_string'].apply(lambda x: float(x.split('_')[5]))
        df['gradient_accumulation_steps'] = df['training_params_string'].apply(lambda x: int(x.split('_')[6]))
        df['learning_rate'] = df['training_params_string'].apply(lambda x: x.split('_')[7].replace('.csv', ''))
    elif df['fine_tuning'][0] in ['base', 'NoFT']:
        df['training_params_string'] = None
    return df

csv_files = glob.glob('data/evaluation_results/*.csv')
dfs = []
empty_n = 0
for file in csv_files:
    if file =='data/evaluation_results/joint_results.csv':
        continue
    tmp_df = pd.read_csv(file)
    if tmp_df.shape[0] == 0:
        print('EMPTY FILE: ', file)
        empty_n += 1
        continue
    print('SOURCE: ', file)
    tmp_df = extract_params_from_file_name(tmp_df)
    dfs.append(tmp_df)

res = pd.concat(dfs)

res = res.drop(['training_params_string', 'similar_is_equal_threshold'], axis = 1)
display(res)
res.to_csv('data/evaluation_results/joint_results.csv')
# res = res.drop(columns=['similar_is_equal', 'similar_is_equal_threshold'])
# print(empty_n)
# res.sort_values(by='f1_score', ascending=False)


SOURCE:  data/evaluation_results/zefiro_4bit_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/mistral_noInstr_8bit_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/zefiro_NoQuant_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/zefiro_8bit_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/qwen_14B_4bit_FT_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/llama_13B_4bit_FT_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/mistral_NoQuant_FT_float16.csv
SOURCE:  data/evaluation_results/llama_7B_8bit_base_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/llama_13B_4bit_base_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/qwen_7B_4bit_FT_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/qwen_7B_4bit_base_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/mistral_NoQuant_base_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/llama_7B_NoQuant_FT_w

,Unnamed: 0,file,f1_score,precision,recall,model_type,model_configurations,model_size,quantization,fine_tuning,...,nShotsInference,model,nbit,bnb_4bit_compute_dtype,r,lora_alpha,lora_dropout,gradient_accumulation_steps,learning_rate,similar_is_equal
0,0.0,data/zefiro/4bit_FT/maxNewTokensFactor8_nShots...,0.400000,0.436886,0.368857,zefiro,4bit_FT,7,4bit,FT,...,4,zefiro-7b-base-ita_,4,torch.bfloat16,32.0,64.0,0.01,4.0,0.0002,NaN
1,1.0,data/zefiro/4bit_FT/maxNewTokensFactor4_nShots...,0.347961,0.420459,0.296788,zefiro,4bit_FT,7,4bit,FT,...,4,zefiro-7b-base-ita_,4,torch.bfloat16,64.0,32.0,0.01,8.0,0.0002,NaN
2,2.0,data/zefiro/4bit_FT/maxNewTokensFactor8_nShots...,0.483046,0.391605,0.630199,zefiro,4bit_FT,7,4bit,FT,...,0,zefiro-7b-base-ita_,4,torch.bfloat16,64.0,32.0,0.01,2.0,0.0002,NaN
3,3.0,data/zefiro/4bit_FT/maxNewTokensFactor8_nShots...,0.414676,0.387327,0.446180,zefiro,4bit_FT,7,4bit,FT,...,0,zefiro-7b-base-ita_,4,torch.bfloat16,32.0,64.0,0.01,4.0,0.0002,NaN
4,4.0,data/zefiro/4bit_FT/maxNewTokensFactor8_nShots...,0.450917,0.394938,0.525387,zefiro,4bit_FT,7,4bit,FT,...,0,zefiro-7b-base-ita_,4,torch.bfloat16,64.0,64.0,0.01,8.0,0.0002,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,NaN,data/qwen/7B_8bit_base/maxNewTokensFactor4_nSh...,0.497369,0.556719,0.449454,qwen,7B_8bit_base,7B,8bit,base,...,3,BaseModel_Qwen1.5-7B-Chat_8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,data/qwen/7B_8bit_base/maxNewTokensFactor8_nSh...,0.520493,0.555185,0.489881,qwen,7B_8bit_base,7B,8bit,base,...,1,BaseModel_Qwen1.5-7B-Chat_8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,data/qwen/7B_8bit_base/maxNewTokensFactor4_nSh...,0.509443,0.575241,0.457153,qwen,7B_8bit_base,7B,8bit,base,...,1,BaseModel_Qwen1.5-7B-Chat_8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,data/qwen/7B_8bit_base/maxNewTokensFactor4_nSh...,0.508342,0.560437,0.465108,qwen,7B_8bit_base,7B,8bit,base,...,4,BaseModel_Qwen1.5-7B-Chat_8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
res

In [ ]:
tmp_df = pd.read_csv(file)
extract_params_from_file_name